In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
import numba as nb
from scipy.optimize import fsolve
from numpy.linalg import inv

In [10]:
# power flow function (equality constraints)
def gfun(x, u, p):
  
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    F1 = 4.0*VM2*VM2 + VM2*VM3*(-4*cos(VA23) + 10*sin(VA23)) - P2
    F2 = (8.0*VM3*VM3 + VM3*VM1*(-4*cos(VA31) + 5*sin(VA31))
          + VM3*VM2*(-4*cos(VA32) + 10*sin(VA32)) + P3)
    F3 = (15.0*VM3*VM3 + VM3*VM1*(-4*sin(VA31) - 5*cos(VA31))
          + VM3*VM2*(-4*sin(VA32) - 10*cos(VA32)) + Q3)

    return np.array([F1, F2, F3])

# cost function
def cfun(x, u, p):

    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    print("P1", 4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13)), "P2", P2)

    cost = (w1*(4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13))) +
          w2*P2)
    
    return cost

In [11]:
# Jacobians and gradients

def gfun_x(x, u, p):

    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  VM2*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[0, 1] =  VM2*VM3*(-4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    J[0, 2] =  VM2*VM3*(4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM1*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3)) + 16.0*VM3
    J[1, 1] =  VM1*VM3*(-4*sin(VA1 - VA3) + 5*cos(VA1 - VA3)) + VM2*VM3*(-4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    J[1, 2] =  VM2*VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM1*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3)) + VM2*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3)) + 30.0*VM3
    J[2, 1] =  VM1*VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[2, 2] =  VM2*VM3*(10*sin(VA2 - VA3) + 4*cos(VA2 - VA3))

    return J

def gfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  0
    J[0, 1] =  -1
    J[0, 2] =  8.0*VM2 + VM3*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[1, 1] =  0
    J[1, 2] =  VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM3*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    J[2, 1] =  0
    J[2, 2] =  VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))

    return J


def cfun_x(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  VM1*w1*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    grad[1] =  VM1*VM3*w1*(-4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    grad[2] =  0
    
    return grad

def cfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  w1*(8.0*VM1 + VM3*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)))
    grad[1] =  w2
    grad[2] =  0
    
    return grad

Initialize script with same initial conditions as in the paper

In [12]:
# initial parameters
x = np.zeros(3)
u = np.zeros(3)
p = np.zeros(3)

# this is an initial guess
x[0] = 1.0 #VM3
x[1] = 0.0 #VA3
x[2] = 0.0 #VA2

# this is given by the problem data, but might be "controlled" via OPF
u[0] = 1.0 #VM1
u[1] = 1.7 #P2
u[2] = 1.0 #VM2

# these parameters are fixed through the computation
p[0] = 0.0 #VA1, slack angle
p[1] = 2.0 #P3
p[2] = 1.0 #Q3

# print initial guesses
print(x)
print(u)

[1. 0. 0.]
[1.  1.7 1. ]


In [5]:
# POWER FLOW ALGO

def powerflow(x, u, p):
    
    sol = fsolve(gfun, x, args=(u,p,))
    return sol
    
print(powerflow(x, u, p))

[ 0.88186783 -0.00094814  0.1349708 ]


In [14]:
# Reduced gradient iteration

max_iter = 50
xk = np.copy(x)
uk = np.copy(u)

for i in range(max_iter):
    
    # power flow
    xk = powerflow(xk, uk, p)

    # lambda calculation
    J_x = gfun_x(xk, uk, p)
    G_x = cfun_x(xk, uk, p)

    lam = -np.dot(inv(np.transpose(J_x)), G_x)
    
    # gradient cost function
    J_u = gfun_u(xk, uk, p)
    G_u = cfun_u(xk, uk, p)
    
    grad_c = G_u + np.dot(np.transpose(J_u), lam)
    print("Norm of gradient: ", np.linalg.norm(grad_c))
    
    # evaluate cost function
    print("Cost function: ", cfun(xk, uk, p))    
    
    # compute step
    alpha = 0.1
    uk = uk - alpha*grad_c

Norm of gradient:  0.7038831243894311
P1 0.47671095119861384 P2 1.7
Cost function:  2.176710951198614
Norm of gradient:  0.3938232615067381
P1 0.46468702735410927 P2 1.691862304561656
Cost function:  2.156549331915765
Norm of gradient:  0.28526493443173734
P1 0.4539008095590158 P2 1.693624129886998
Cost function:  2.147524939446014
Norm of gradient:  0.24055914798927536
P1 0.4503136349680572 P2 1.6906811085664968
Cost function:  2.140994743534554
Norm of gradient:  0.22011394725879266
P1 0.44588920332688264 P2 1.6897654075062027
Cost function:  2.135654610833085
Norm of gradient:  0.20668282032924015
P1 0.4431199815222504 P2 1.687877642534445
Cost function:  2.1309976240566955
Norm of gradient:  0.1959223286707491
P1 0.4404903162224345 P2 1.6863553171891814
Cost function:  2.126845633411616
Norm of gradient:  0.18663816069900974
P1 0.43845527819007746 P2 1.6846450658234962
Cost function:  2.1231003440135736
Norm of gradient:  0.17841638216748473
P1 0.43670109148821634 P2 1.682993673702